## 生成随机数

In [1]:
import java.util.concurrent.ThreadLocalRandom
import breeze.linalg._
import scala.math._
import breeze.stats.distributions._
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.mllib.linalg.{Vectors,Matrix => sparkMatrix,DenseMatrix=> sparkDenseMatrix}

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-9RG91OPS:4041
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1676720647723)
SparkSession available as 'spark'


import java.util.concurrent.ThreadLocalRandom
import breeze.linalg._
import scala.math._
import breeze.stats.distributions._
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.mllib.linalg.{Vectors, Matrix=>sparkMatrix, DenseMatrix=>sparkDenseMatrix}


In [2]:
val N=sc.broadcast(1000)
val p=sc.broadcast(200)
val maxIter=sc.broadcast(6)
val Beta=sc.broadcast(DenseVector((1 to p.value).map(_.toDouble).map(s=> {
    if (s<=3) 2.0 else 0.0
}).toArray))
val Indeces=0 until N.value

N: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(0)
p: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(1)
maxIter: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(2)
Beta: org.apache.spark.broadcast.Broadcast[breeze.linalg.DenseVector[Double]] = Broadcast(3)
Indeces: scala.collection.immutable.Range = Range 0 until 1000


In [3]:
def tobreezematrix(tmp1:sparkMatrix):DenseMatrix[Double]={
    var tmp2=DenseMatrix.zeros[Double](tmp1.numRows,tmp1.numCols)
    for (i<- 0 until tmp1.numRows){
        for(j <- 0 until tmp1.numCols){
            tmp2(i,j)=tmp1(i,j)
        }
    }
    tmp2
}

tobreezematrix: (tmp1: org.apache.spark.mllib.linalg.Matrix)breeze.linalg.DenseMatrix[Double]


In [4]:
val ParalIndeces=sc.parallelize(Indeces)
var sample_matrix=ParalIndeces.map(s => {
    var norm_dist=new Gaussian(0,1)
    var x=new DenseMatrix(rows=1,cols=p.value,norm_dist.sample(p.value).toArray)
    var y=x*Beta.value+norm_dist.draw()
    Array(s.toDouble).union(x.toArray).union(Array(y(0)))
})
sample_matrix.persist()
var sample_x=sample_matrix.map(f => {
    IndexedRow(f.take(1)(0).toLong,Vectors.dense(f.drop(1).dropRight(1)))
})
var sample_y=sample_matrix.map(f => {
    IndexedRow(f.take(1)(0).toLong,Vectors.dense(f.drop(p.value+1)))
})
var sample_x_indexedrowmatrix=new IndexedRowMatrix(sample_x)
var sample_y_indexedrowmatrix=new IndexedRowMatrix(sample_y)
var block_x=sample_x_indexedrowmatrix.toBlockMatrix(rowsPerBlock=N.value/10,colsPerBlock=p.value/10)
var block_y=sample_y_indexedrowmatrix.toBlockMatrix(rowsPerBlock=N.value/10,colsPerBlock=1)
val X=tobreezematrix(block_x.toLocalMatrix)
val y=tobreezematrix(block_y.toLocalMatrix).toDenseVector
val inibetahat=inv(X.t*X)*X.t*y
var betahat=DenseVector.zeros[Double](p.value)

23/02/18 19:44:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/02/18 19:44:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


ParalIndeces: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at <console>:45
sample_matrix: org.apache.spark.rdd.RDD[Array[Double]] = MapPartitionsRDD[1] at map at <console>:46
sample_x: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.distributed.IndexedRow] = MapPartitionsRDD[2] at map at <console>:53
sample_y: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.distributed.IndexedRow] = MapPartitionsRDD[3] at map at <console>:56
sample_x_indexedrowmatrix: org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix = org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix@f59bcc5
sample_y_indexedrowmatrix: org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix = org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix@3c268372
block_x: o...


In [5]:
//另一种更简单的随机数生成方式：
//val x = new DenseMatrix(rows = N.value, cols = P.value, norm_dist.sample(N.value*P.value).toArray)
//val epsilon=(new DenseMatrix(rows=N.value,cols=1,norm_dist_epsilon.sample(N.value).toArray)).toDenseVector
//val y=x * beta +epsilon

## 第一问：串型程序

In [6]:
betahat(0 until p.value):=inibetahat(0 until p.value)
val lam=sc.broadcast(0.85e-12)
var ii=0
var Diff=1.0
val eps=sc.broadcast(1e-5)
var Oldbeta=DenseVector.zeros[Double](p.value)
while(ii<maxIter.value && Diff>eps.value){
    Oldbeta(0 until p.value):=betahat(0 until p.value)
    for(i<- 0 until p.value){
        if (math.abs(X(::,i).t*(y-X*Oldbeta))<lam.value) {
            betahat(i)=0.0
        }
        else {
            val tmp=DenseVector.zeros[Double](N.value)
            for(j<-0 until p.value){
                if(j!=i) tmp+=X(::,j)*Oldbeta(j)
            }
            betahat(i)=X(::,i).t*(y-tmp)/(lam.value/math.abs(Oldbeta(i))+X(::,i).t*X(::,i))
        }
    }
    Diff=sum((Oldbeta-betahat).map(math.abs(_)))
    println(Diff)
    ii+=1
}
println(betahat)

5.1925298972254135
0.06930251059112796
8.334467352879363E-4
3.201938074665067E-5
3.0197561196806944E-6
DenseVector(2.0323496344612733, 1.9915905493814676, 2.0358792666521106, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.04057263816221226, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

lam: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(12)
ii: Int = 5
Diff: Double = 3.0197561196806944E-6
eps: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(13)
Oldbeta: breeze.linalg.DenseVector[Double] = DenseVector(2.032349136856064, 1.9915909962929539, 2.035879390492781, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.040570686763458486, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0,...


## 第二问：并行程序

有办法解除+0和-0的

In [7]:
betahat(0 until p.value):=inibetahat(0 until p.value)
val lam=sc.broadcast(0.85e-12)
var ii=0
var Diff=1.0
val eps=sc.broadcast(1e-5)
var Oldbeta=DenseVector.zeros[Double](p.value)
var x_array=(0 to p.value-1).map(s => (s,X(::,s),betahat(s))).toArray  //确实可以事后给他打标签
var x=sc.parallelize(x_array)
var xs=DenseVector.zeros[Double](N.value)
while(ii<maxIter.value && Diff>eps.value){
    Oldbeta(0 until p.value):=betahat(0 until p.value) //这样才能达到拷贝的效果
    xs(0 until N.value):=x.map(s => {
        s._2*s._3
    }).reduce((x,y)=>x+y)
    val tmp=x.map(s => {
        if(math.abs(s._2.t*(y-X*Oldbeta))<lam.value){
            0.0
        }
        else {
            s._2.t*(y-(xs-s._2*Oldbeta(s._1)))/(lam.value/math.abs(Oldbeta(s._1))+s._2.t*s._2)
        }
    }).collect()
    betahat(0 until p.value):=DenseVector(tmp)
    x_array=(0 to p.value-1).map(s => (s,X(::,s),betahat(s))).toArray
    x=sc.parallelize(x_array)
    Diff=sum((Oldbeta-betahat).map(math.abs(_)))
    ii+=1
}
println(betahat)

23/02/18 19:44:42 WARN TaskSetManager: Stage 8 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 19:44:42 WARN DAGScheduler: Broadcasting large task binary with size 1604.2 KiB
23/02/18 19:44:42 WARN TaskSetManager: Stage 9 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 19:44:42 WARN TaskSetManager: Stage 10 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 19:44:43 WARN DAGScheduler: Broadcasting large task binary with size 1604.2 KiB
23/02/18 19:44:43 WARN TaskSetManager: Stage 11 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 19:44:43 WARN TaskSetManager: Stage 12 contains a task of very large size (1568 KiB). The maximum recommended task size is 1000 KiB.
23/02/18 19:44:43 WARN DAGScheduler: Broadcasting large task binary with size 1604.2 KiB
23/02/18 19:44:43 WARN Ta

lam: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(14)
ii: Int = 5
Diff: Double = 3.0197561192782385E-6
eps: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(15)
Oldbeta: breeze.linalg.DenseVector[Double] = DenseVector(2.0323491368560647, 1.9915909962929539, 2.035879390492781, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.040570686763458486, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0...


23/02/18 23:26:38 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1053)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:238)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java